
... ***CURRENTLY UNDER DEVELOPMENT*** ...


## Obtain wave families
### In this case, two swell systems, one local sea

inputs required: 
  * Historical wave conditions (GOW2 hindcast)
  * Wave families sectors; the split is based on wave direction
  
in this notebook:
  * Split energy based on defined sectors
  * Remove TC associated waves to avoid double counting 


### Workflow:

<div>
<img src="resources/nb01_09.png" width="300px">
</div>


In [13]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op
from datetime import datetime

# pip
import numpy as np
import xarray as xr

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.waves import GetDistribution_gow, GetDistribution_ws

# wavespectra

from wavespectra.specdataset import SpecDataset



## Database and Site parameters

In [14]:
# --------------------------------------
# Teslakit database

p_data = r'/media/administrador/HD/Dropbox/Guam/teslakit/data'
#p_data=r'/Users/laurac/Dropbox/Guam/teslakit/data'
db = Database(p_data)

# set site
db.SetSite('GUAM')


In [28]:
# --------------------------------------
# set waves families parameters

_, TCs_r1_params = db.Load_TCs_r1_hist()   # TCs historical parameters inside big radius
_, TCs_r2_params = db.Load_TCs_r2_hist()   # TCs historical parameters inside small radius

# wave families sectors
#fams_sectors = [(315, 45), (45, 202.5), (202.5, 315)]
fams_sectors = [(202.5, 60), (60, 202.5)]

# date limits for TCs removal from waves data, and TC time window (hours)
tc_rm_date1 = '1979-01-01'
tc_rm_date2 = '2019-12-31'
tc_time_window = 12



## Calculate Waves Partitions from Waves Spectra (CSIRO + wavespectra) 

In [4]:
# aux.

def fix_dir(base_dirs):
    'fix csiro direction for wavespectra (from -> to)'
    new_dirs = base_dirs + 180
    new_dirs[np.where(new_dirs>=360)] = new_dirs[np.where(new_dirs>=360)] - 360
    
    return new_dirs


In [ ]:
# --------------------------------------
# load waves spectra point (CSIRO spec)

WVS_spec = db.Load_WAVES_spectra()
print(WVS_spec)
print()

# direction data fix
# WVS_spec['direction'] = fix_dir(WVS_spec['direction'])

# # rename variables
# WVS_spec = WVS_spec.rename(
#     {
#         'frequency':'freq',
#         'direction':'dir',
#         'Efth':'efth',
#     }
# ).set_coords({'freq','dir'})

# efth: rad to º
# WVS_spec['efth'] = WVS_spec['efth'] #* np.pi/180


# wavespectra parameters
wcut = 0.00000000001  # wcut = 0.3333
msw = 8
agef = 1.7

# bulk wavespectra
bulk_params = WVS_spec.spec.stats(['hs','tp','tm02','dpm','dspr'])

# partitions
ds_part = WVS_spec.spec.partition(
    WVS_spec.Wspeed, WVS_spec.Wdir, WVS_spec.Depth, 
    wscut = wcut, max_swells = msw, agefac = agef,
)

WVS_parts = ds_part.spec.stats(['hs','tp','tm02','dpm','dspr']) 

# Add bulk Hs, Tp, Dir variables
WVS_parts['Hs'] = bulk_params['hs']
WVS_parts['Tp'] = bulk_params['tp']
WVS_parts['Dir'] = bulk_params['dpm']

# drop station id
# WVS_parts = WVS_parts.drop('station')

# # Save partitions data
db.Save_WAVES_partitions(WVS_parts)

<xarray.Dataset>
Dimensions:  (dir: 24, freq: 29, time: 365756)
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-01-01T01:00:00 ... 2020-09-01
  * dir      (dir) float32 262.5 247.5 232.5 217.5 ... 322.5 307.5 292.5 277.5
  * freq     (freq) float32 0.035 0.0385 0.042349998 ... 0.45885003 0.50473505
Data variables:
    efth     (time, freq, dir) float64 ...
    Wspeed   (time) float32 ...
    Wdir     (time) float32 ...
    Depth    (time) float32 ...



In [ ]:
print(WVS_parts)


## Calculate Historical Waves Families (CSIRO + wavespectra)

In [29]:
# --------------------------------------
# Calculate wave families from waves partitions data and waves sectors

WVS_parts = db.Load_WAVES_partitions()       # waves partitions data (from CSIRO spectra and wavespectra toolbox)

WVS = GetDistribution_ws(WVS_parts, fams_sectors, n_partitions=8)

# Add wavespectra bulk Hs, Tp, Dir variables
WVS['Hs'] = WVS_parts['Hs']
WVS['Tp'] = WVS_parts['Tp']
WVS['Dir'] = WVS_parts['Dir']

# ensure time dimension does not repeat values
_, index = np.unique(WVS['time'], return_index=True)
WVS = WVS.isel(time=index)

print(WVS)


<xarray.Dataset>
Dimensions:      (time: 365257)
Coordinates:
  * time         (time) datetime64[ns] 1979-01-01 ... 2020-09-01
Data variables:
    sea_Hs       (time) float64 nan 0.3987 0.5262 0.6879 ... nan nan 1.033 1.028
    sea_Tp       (time) float64 nan 2.537 2.946 3.458 ... nan nan 7.648 7.694
    sea_Dir      (time) float64 nan 74.86 74.6 73.9 ... nan nan 99.93 99.97
    swell_1_Hs   (time) float64 nan nan nan nan nan ... 1.64 1.633 1.622 1.61
    swell_1_Tp   (time) float64 nan nan nan nan nan ... 11.26 11.24 11.25 11.3
    swell_1_Dir  (time) float64 nan nan nan nan nan ... 275.1 275.0 275.0 275.0
    swell_2_Hs   (time) float64 nan nan nan nan ... 1.091 1.087 0.3185 0.3142
    swell_2_Tp   (time) float64 nan nan nan nan nan ... 8.281 8.329 12.45 12.48
    swell_2_Dir  (time) float64 nan nan nan nan nan ... 101.6 101.6 73.77 73.03
    Hs           (time) float64 ...
    Tp           (time) float64 ...
    Dir          (time) float64 ...



## Calculate Historical Waves Families (GOW)

In [17]:
# --------------------------------------
# Calculate wave families from waves partitions data and waves sectors

#WVS_pts = db.Load_WAVES_partitions_GOW()  # waves partitions data (GOW)

#WVS = GetDistribution_gow(WVS_pts, fams_sectors, n_partitions=5)

# Add GOW Hs, Tp, Dir variables
#WVS['Hs'] = WVS_pts['hs']
#WVS['Tp'] = WVS_pts['tp']
#WVS['Dir'] = WVS_pts['dir']



## TCs: Waves  Selection 

In [33]:
# --------------------------------------
# Locate TCs and set category alongside WAVES data  

# remove TCs before 1979 and after 2020 (r1)
dds = TCs_r1_params.dmin_date.values[:]
ix = np.where((dds >= np.datetime64(tc_rm_date1)) & (dds <= np.datetime64(tc_rm_date2)))[0]
TCs_r1_params = TCs_r1_params.isel(storm=ix)

# select storms inside big circle 
storms_sel = TCs_r1_params.storm.values[:]

# add TCs category alongside WAVES data
WVS['TC_category'] = (('time',), np.empty(len(WVS.time))*np.nan)

for s in storms_sel:

    # waves at storm dates
    ss = TCs_r1_params.sel(storm=s)     
    wvs_s = WVS.sel(time = slice(ss.dmin_date, ss.last_date))
    
    # get hs_max date 
    t_hs_max = wvs_s.where(wvs_s.Hs ==  wvs_s.Hs.max(), drop=True).time.values[:][0]
        
    # hs_max time window 
    w1 = t_hs_max - np.timedelta64(tc_time_window,'h')
    w2 = t_hs_max + np.timedelta64(tc_time_window,'h')
    
    # set category alongside WAVES data
    ixs = np.where((WVS.time >= w1) & (WVS.time <= w2))[0]
    WVS['TC_category'][ixs] = ss.category 
    
print(WVS)

# Store historical WAVES data
db.Save_WAVES_hist(WVS)


<xarray.Dataset>
Dimensions:      (time: 365257)
Coordinates:
  * time         (time) datetime64[ns] 1979-01-01 ... 2020-09-01
Data variables:
    sea_Hs       (time) float64 nan 0.3987 0.5262 0.6879 ... nan nan 1.033 1.028
    sea_Tp       (time) float64 nan 2.537 2.946 3.458 ... nan nan 7.648 7.694
    sea_Dir      (time) float64 nan 74.86 74.6 73.9 ... nan nan 99.93 99.97
    swell_1_Hs   (time) float64 nan nan nan nan nan ... 1.64 1.633 1.622 1.61
    swell_1_Tp   (time) float64 nan nan nan nan nan ... 11.26 11.24 11.25 11.3
    swell_1_Dir  (time) float64 nan nan nan nan nan ... 275.1 275.0 275.0 275.0
    swell_2_Hs   (time) float64 nan nan nan nan ... 1.091 1.087 0.3185 0.3142
    swell_2_Tp   (time) float64 nan nan nan nan nan ... 8.281 8.329 12.45 12.48
    swell_2_Dir  (time) float64 nan nan nan nan nan ... 101.6 101.6 73.77 73.03
    Hs           (time) float64 ...
    Tp           (time) float64 ...
    Dir          (time) float64 ...
    TC_category  (time) float64 nan nan